In [1]:
!pip install tqdm
!pip install langchain
!pip install langchain-community
!pip install langchain-qdrant
!pip install gpt4all
!pip install pypdf

  Using cached langchain_qdrant-0.2.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached qdrant_client-1.13.2-py3-none-any.whl.metadata (10 kB)
  Using cached grpcio_tools-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached grpcio-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached langchain_qdrant-0.2.0-py3-none-any.whl (23 kB)
Using cached qdrant_client-1.13.2-py3-none-any.whl (306 kB)
Using cached grpcio_tools-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
Using cached grpcio-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
   

In [2]:
import os
from glob import glob

from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_qdrant import QdrantVectorStore
from dotenv import load_dotenv
import boto3

In [3]:
load_dotenv()

True

In [4]:
from sagemaker import get_execution_role

get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'arn:aws:iam::409029738116:role/service-role/AmazonSageMaker-ExecutionRole-20250213T110935'

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=20,
    length_function=len
)

In [6]:
def get_pages_from_pdf_document(pdffilepath):
    loader = PyPDFLoader(pdffilepath)
    pages = loader.load_and_split(text_splitter=text_splitter)
    return pages


def get_pages_from_pdf_documents(directory):
    pages = []
    for pdffilepath in tqdm(glob(os.path.join(directory, '*.pdf'))):
        this_pages = get_pages_from_pdf_document(pdffilepath)
        if this_pages is not None and len(this_pages) > 0:
            pages += this_pages
    return pages


def iterate_list_pdfnames_localdir(directory):
    for pdffilepath in tqdm(glob(os.path.join(directory, '*.pdf'))):
        basename = os.path.basename(pdffilepath)
        yield basename


def iterate_list_pdfnames_s3(bucket_name, folder):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    for file in bucket.objects.all():
        dirname = os.path.dirname(file.key)
        if dirname == folder:
            basename = os.path.basename(file.key)
            _, ext = os.path.splitext(basename)
            if ext == '.pdf':
                yield basename


def get_pages_from_pdf_documents_in_s3(bucket_name, folder):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    pages = []
    for filebasename in tqdm(iterate_list_pdfnames_s3(bucket_name, folder)):
        copiedfilepath = os.path.join('/home/sagemaker-user', filebasename)
        bucket.download_file(os.path.join(folder, filebasename), copiedfilepath)
        this_pages = get_pages_from_pdf_document(copiedfilepath)
        if this_pages is not None and len(this_pages) > 0:
            pages += this_pages
        os.remove(copiedfilepath)
    return pages

In [7]:
def main(bucket_name, folder, embed_model_name, qdrant_url, qdrant_api_key):
    embedding_model = GPT4AllEmbeddings(model_name=embed_model_name)
    pages = get_pages_from_pdf_documents_in_s3(bucket_name, folder)
    qdrant = QdrantVectorStore.from_documents(
        pages,
        embedding_model,
        url=qdrant_url,
        prefer_grpc=True,
        api_key=qdrant_api_key,
        collection_name='mlbooks'
    )

In [8]:
corpusdir = 's3://mlbooksragbucket/corpus'
embed_model_name = 'all-MiniLM-L6-v2.gguf2.f16.gguf'

qdrant_url = os.getenv('QDRANT_URL')
qdrant_api_key = os.getenv('QDRANT_API_KEY')

In [9]:
f = open(os.path.join('/home/sagemaker-user', 'booklist_qdrant.txt'), 'w')
for line in iterate_list_pdfnames_s3('mlbooksragbucket', 'corpus'):
  f.write(line + '\n')
f.close()

In [10]:
main('mlbooksragbucket', 'corpus', embed_model_name, qdrant_url, qdrant_api_key)

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
16it [01:33,  6.30s/it]Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 229 0 (offset 0)
Ignoring wrong pointing object 642 0 (offset 0)
Ignoring wrong pointing object 644 0 (offset 0)
32it [03:54,  5.36s/it]Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointi

UnicodeEncodeError: 'utf-8' codec can't encode character '\ud835' in position 215: surrogates not allowed